Так
Я хочу штуки блоками?
Как правильно — передавать штуки или не передавать штуки?

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import re
import time

# today's wordle

from bs4 import BeautifulSoup
import requests

class PasswordLetter:

    def __init__(self, letter: str) -> None:
        self.letter = letter # wtf naming lol I dont like it
        self.bold = False
        self.italic = False
        self.font_size = None
        self.font = None

    def to_html(self):
        result = self.letter
        if self.bold:
            result = f"<strong>{result}</strong>"    
        if self.italic:
            result = f"<i>{result}</i>"
        #<span style="font-family: Wingdings; font-size: 28px">HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH</span>   
        if self.font_size is not None or self.font is not None:
            self.font_size = '28' if self.font_size is None else self.font_size
            self.font = 'Monospace' if self.font is None else self.font
            result = f'<span style="font-family: {self.font}; font-size: {self.font_size}px">{result}</span>'
        return result

def password_to_str(password: list[PasswordLetter]):
    return ''.join([letter.to_html() for letter in password])

def password_to_str_wo_html(password: list[PasswordLetter]):
    return ''.join([letter.letter for letter in password])

def str_to_password(string: str):
    return [PasswordLetter(letter) for letter in string]

def replace_text_in_password_box(password: list[PasswordLetter]):
    driver.execute_script("document.querySelector('.ProseMirror').innerHTML = '<p>{}</p>';".format(password_to_str(password))) #actually faster than sendkeys()


def today_wordle():
    url_wordle_ans = ' https://word.tips/todays-wordle-answer/'

    # today date
    import datetime
    today_date = datetime.datetime.now()
    today_worlde_raw = today_date.strftime("%B %-d") # July 4
    page = requests.get(url_wordle_ans, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(page.content, "html.parser")
    child_soup = soup.find_all('strong')
    for x in child_soup:
        if (today_worlde_raw in x.string):
            return x.string[-5:].lower()
        
def today_moon_phase():
    #return '🌖'
    dict_moon_phases = {
        'New' : '🌑',
        'Waxing Crescent' : '🌒',
        'First Quarter' : '🌓',
        'Waxing Gibbous' : '🌔',
        'Full' : '🌕',
        'Waning Gibbous' : '🌖',
        'Last Quarter' : '🌗',
        'Waning Crescent' : '🌘'
    } # should I have it like this?

    url_moon_phases = 'https://www.moongiant.com/phase/today/'
    page = requests.get(url_moon_phases, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(page.content, "html.parser")
    moon_details = soup.find("div", {"id": "moonDetails"})
    return dict_moon_phases[moon_details.contents[1].string]       

def strong():
     return '🏋️‍♂️🏋️‍♂️🏋️‍♂️'

url_password_game = 'https://neal.fun/password-game/' # global
# free_digit = 25 # do I need that?
# password_digits = [1 for _ in range(25)] # digits that is not in captcha or chess or whatever; do I need that?

first_password = '1' * 25 + '$0XXXVpepsimayHe iamloved' + today_wordle() + today_moon_phase() + strong() # add leap year in the beginning

password = str_to_password(first_password)

# start browser operation

# driver = webdriver.Chrome() 
driver = webdriver.Safari()
driver.get(url_password_game)
time.sleep(1)
password_box = driver.find_element(By.CLASS_NAME, 'ProseMirror')

replace_text_in_password_box(password) #password_box.send_keys(password)

captcha_img = driver.find_element(By.CLASS_NAME, 'captcha-img').get_attribute('src')
refresh_captcha = driver.find_element(By.CLASS_NAME, 'captcha-refresh')

#captchasolver
def captcha_solver():
    captcha_digit_sum = 30
    while captcha_digit_sum > 5: # magic int; I think it will be enough degree of freedom
        captcha_img = driver.find_element(By.CLASS_NAME, 'captcha-img').get_attribute('src')
        captcha = captcha_img[40:45]
        # refaсtoring digits in password if captcha have some to sum to 25
        captcha_digit_sum = 0
        for x in captcha:
            if x.isdigit():
                captcha_digit_sum += int(x)
        if captcha_digit_sum > 5:
            time.sleep(1)
            refresh_captcha.click()
    return captcha, captcha_digit_sum

captcha, captcha_digit_sum = captcha_solver()
password = (password[captcha_digit_sum:] if captcha_digit_sum != 0 else password) + str_to_password(captcha)  # I had a bug if captcha were without digit lol

# password_box.clear()
replace_text_in_password_box(password) #password_box.send_keys(password)      

countiers = pd.read_json('maps.jsonc')

# Just could not do anything with them
# geo_frame = driver.find_element(By.XPATH, r'//*[@id="__layout"]/div/div/div[2]/div[5]/div/div[1]/div/div/div/div[2]/div/iframe')
# geo = driver.find_element(By.CLASS_NAME, 'geo')

def geo_solver(page_source_geo):
    soup = BeautifulSoup(page_source_geo, "html.parser")
    child_soup = soup.find_all('iframe')
    for x in child_soup:
        if x['src'].startswith("https://www.google.com/maps/embed"):
            geo_embed = x['src']
    return countiers[countiers.embed == geo_embed].title.values[0].lower().replace(" ", "")

country = geo_solver(driver.page_source)
password = password + str_to_password(country)
replace_text_in_password_box(password) #password_box.send_keys(country)  

def get_chess_position(chess_img):
    import requests
    from xml.etree import ElementTree as ET

    # Make a request to the URL
    response = requests.get(chess_img)

    # Check if the request was successful
    if response.status_code == 200:
        svg_data = response.content

        # Parse SVG data
        tree = ET.ElementTree(ET.fromstring(svg_data))

        # Loop over all elements in the SVG
        for elem in tree.iter():
            if elem.text is not None:
                chess_position = elem.text
                return chess_position
    else:
        print('Failed to fetch SVG.') # TODO raise Err
    return

def color_move(chess_move):
    if 'White' in chess_move:
        return 'w'
    if 'Black' in chess_move:
        return 'b'
    return 'WRONG MOVE'

def from_svg_to_FEN(chess_position, chess_move):
    # Remove leading/trailing white space and split the string into lines
    rows = chess_position.strip().split("\n")

    # Initialize an empty list to hold FEN rows
    fen_rows = []

    # Iterate over rows
    for row in rows:
        # Remove leading/trailing white space and split row into squares
        squares = row.strip().split(" ")
        # Initialize counter for empty squares
        empty = 0
        # Initialize an empty string to hold FEN for this row
        fen_row = ''
        # Iterate over squares
        for square in squares:
            if square == '.':
                # Increment counter for empty squares
                empty += 1
            else:
                # If the square is not empty and there were empty squares before, add count to FEN
                if empty > 0:
                    fen_row += str(empty)
                    # Reset empty squares counter
                    empty = 0
                # Add piece to FEN
                fen_row += square
        # If the row ends with one or more empty squares, add count to FEN
        if empty > 0:
            fen_row += str(empty)
        # Add FEN for this row to the list of FEN rows
        fen_rows.append(fen_row)
    # Join the FEN rows with slashes to get the final FEN
    fen_board = '/'.join(fen_rows)
    fen = fen_board + ' ' + color_move(chess_move) + ' - - 0 1'

    return fen

def get_best_move(fen): 

    import chess.engine

    # Define the path to your Stockfish engine file
    engine_path = "/opt/homebrew/Cellar/stockfish/16/bin/stockfish"

    # Initialize the chessboard with a given FEN
    board = chess.Board(fen)

    # Initialize the engine
    engine = chess.engine.SimpleEngine.popen_uci(engine_path)

    # Find the best move
    result = engine.play(board, chess.engine.Limit(time=1.0))
    best_move = result.move

    # From UCI to algebraic notation
    best_move_alg_not = board.san(best_move)

    # It is good practice to close the engine after using it
    engine.quit()

    return best_move_alg_not

def solve_chess_position(chess_img, chess_move):
    chess_position = get_chess_position(chess_img)
    fen = from_svg_to_FEN(chess_position, chess_move)
    best_move = get_best_move(fen)
    return best_move

chess_img = driver.find_element(By.CLASS_NAME, 'chess-img').get_attribute('src')
chess_move = driver.find_element(By.CLASS_NAME, 'move').text.strip()
chess_solution = solve_chess_position(chess_img, chess_move)
chess_digit_sum = sum([int(x) for x in re.findall(r'\d', chess_solution)])
password = str_to_password('🥚') + (password[chess_digit_sum:] if chess_digit_sum != 0 else password) + str_to_password(chess_solution)
#password_box.clear()
replace_text_in_password_box(password) #password_box.send_keys(password) 

import pandas as pd
df_atomics = pd.read_csv('atomic_numbers.csv', sep=';')
dict_atomics = dict(zip(df_atomics.symbol, df_atomics.number))
dict_numbers = dict(zip(df_atomics.number, df_atomics.symbol)) # TODO rename that shit

def detect_elements(s):
    i = 0
    detected_list = []
    two_symbols_flg = 0
    candidate = ''
    while i < len(s):
        if s[i].isupper():
            if i + 1 == len(s):
                candidate = s[i]
            elif s[i+1].islower():
                two_symbols_flg = 1
                candidate = s[i:i+2]
            else:
                candidate = s[i]
            if candidate in dict_atomics.keys():
                detected_list.append(candidate)
                i += two_symbols_flg
            elif two_symbols_flg == 1:
                if candidate[:1] in dict_atomics.keys():
                    detected_list.append(candidate[:1])
        two_symbols_flg = 0
        i += 1
    return detected_list

def add_periodic_elements(curr_elements):
    result_list = []
    curr_sum = sum([dict_atomics[elem] for elem in curr_elements])
    if curr_sum > 200:
        print('We fucked up')
        return # TODO raise exception
    result_list.append('H' * (200 - curr_sum))
    return result_list

elements_in_password = add_periodic_elements(detect_elements(password_to_str_wo_html(password)))
password = password + str_to_password(''.join(elements_in_password))
replace_text_in_password_box(password) #password_box.send_keys(''.join(elements_in_password)) 

# I think I need two copy of the password (simple and formatted) to keep track what is going on orrrr just do formating every time from scratch?

def bold_formatting(password: list[PasswordLetter]) -> str: # rule 20
    for letter in password:
        if letter.letter in 'aeiouyAEIOUY':
            letter.bold = True
    return password

password = bold_formatting(password)
replace_text_in_password_box(password)
replace_text_in_password_box(password)

password = str_to_password('🐔🐛🐛🐛🐛') + password[1:]
replace_text_in_password_box(password)

def youtube_solver(page_source):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    child_soup = soup.find_all(class_='rule rule rule-error youtube')
    x = child_soup[0]
    exact_time = re.findall('Your password must include the URL of an exactly \d+ minute long YouTube video', x.text)
    if len(exact_time) > 0:
        duration_list = re.findall(r'\d+', exact_time) + ['0']
    else:
        duration_list = re.findall(r'\d+', re.findall('Your password must include the URL of a \d+ minute \d+ second long YouTube video', x.text)[0])
    duration = int(duration_list[0]) * 60 + int(duration_list[1])

    return duration_list, 'youtu.be/' + youtube_cheatsheet[youtube_cheatsheet.duration == duration].url.values[0]

youtube_cheatsheet = pd.read_csv('youtube_cheatsheet.csv')
duration_list, youtube_url = youtube_solver(driver.page_source)
youtube_url_sum = sum([int(x) for x in re.findall(r'\d', youtube_url)])
password = password[:5] + password[5 + youtube_url_sum:]
youtube_url_elements_sum = sum([dict_atomics[elem] for elem in detect_elements(youtube_url)])
password = (password[:-youtube_url_elements_sum] if youtube_url_elements_sum != 0 else password)

password = password + str_to_password(youtube_url)
password = bold_formatting(password)
replace_text_in_password_box(password)

import string
absence_eng_letter = sorted(list(set(string.ascii_lowercase)-set(password_to_str_wo_html(password).lower())), reverse=True)

scr_letter = (ord(absence_eng_letter[0]) - 96, ord(absence_eng_letter[1]) - 96) # TODO exceptions

# logging
from csv import writer
from datetime import datetime

yt_ok = True
try:
    driver.find_element(By.XPATH, f'//*[@id="__layout"]/div/div/div[2]/div[5]/div/div[1]/div/div/div/div[2]/div/button[{scr_letter[0]}]').send_keys(Keys.ENTER) # get rid of w
except:
    yt_ok = False


# List that we want to add as a new row
log_list = [datetime.now(), password_to_str_wo_html(password), captcha, country, chess_img, chess_move, chess_solution,
            duration_list, youtube_url, ''.join(absence_eng_letter), scr_letter, yt_ok]

# Open our existing CSV file in append mode
# Create a file object for this file
with open('logs.csv', 'a') as f_object:

	# Pass this file object to csv.writer()
	# and get a writer object
	writer_object = writer(f_object)

	# Pass the list as an argument into
	# the writerow()
	writer_object.writerow(log_list)

	# Close the file object
	f_object.close()


time.sleep(3)
# rule 25
# there is some urls which contains w, need to be changed
driver.find_element(By.XPATH, f'//*[@id="__layout"]/div/div/div[2]/div[5]/div/div[1]/div/div/div/div[2]/div/button[{scr_letter[0]}]').send_keys(Keys.ENTER) # get rid of w
# and z. fuck
driver.find_element(By.XPATH, f'//*[@id="__layout"]/div/div/div[2]/div[5]/div/div[1]/div/div/div/div[2]/div/button[{scr_letter[1]}]').send_keys(Keys.ENTER) # get rid of z
time.sleep(3) # overwise smth goes wrong
driver.find_element(By.CLASS_NAME, 'sacrafice-btn').send_keys(Keys.ENTER) # SACRAFICEEEE


def italic_formatting(password: list[PasswordLetter]) -> list[PasswordLetter]:
    bold_cnt = sum([letter.bold for letter in password])
    for i in range(bold_cnt*2):
        password[i].italic = True
    return password

password = italic_formatting(password)
replace_text_in_password_box(password)

#Wingdings
def wingdings_formatting(password: list[PasswordLetter]) -> list[PasswordLetter]:
    wingdings_cnt = len(password) // 3 + 1
    i = 5
    curr_w_cnt = 0
    while curr_w_cnt <= wingdings_cnt:
        if password[i].letter not in 'IVX':
            password[i].font = 'Wingdings'
            curr_w_cnt += 1
        i += 1
    return password

password = wingdings_formatting(password)
replace_text_in_password_box(password)

#driver.quit()

In [15]:
driver.quit()


In [2]:
password_to_str_wo_html(password)

'🐔🐛🐛🐛🐛1111111111111111$0XXXVpepsimayHe iamlovedhobby🌒🏋️\u200d♂️🏋️\u200d♂️🏋️\u200d♂️x3fwfnetherlandsRxf6+HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHyoutu.be/ze-yx_Zgpec'

In [4]:
password_to_str(password)

'<i>🐔</i><i>🐛</i><i>🐛</i><i>🐛</i><i>🐛</i><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span style="font-family: Wingdings; font-size: 28px><i>1</i></span><span s

In [8]:
with open('event.csv', 'a') as f_object:

	# Pass this file object to csv.writer()
	# and get a writer object
	writer_object = writer(f_object)

	# Pass the list as an argument into
	# the writerow()
	writer_object.writerow(log_list)

	# Close the file object
	f_object.close()

In [127]:
password = italic_formatting(password)
replace_text_in_password_box(password)

In [87]:
driver.find_element(By.CLASS_NAME, 'sacrafice-btn').send_keys(Keys.ENTER)


In [3]:
replace_text_in_password_box(password)

In [31]:
driver.maximize_window()

In [2]:
first_sacriface = ord('z') - 97

In [3]:
first_sacriface

25

In [83]:
driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div/div[2]/div[6]/div/div[1]/div/div/div/div[2]/button').send_keys(Keys.ENTER)

NoSuchElementException: Message: ; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception


Your password must include the URL of a 5 minute 34 second long YouTube video.

In [64]:
#captchasolver

from twocaptcha import TwoCaptcha

solver = TwoCaptcha('998379d29f5af42b1d99b164ef2cc74f')
result = solver.normal(captcha_img)
captcha = result['code']

!pip3 install 2captcha-python

3 minute 42 seconds fails
17 minutes fails
23 minute 50 second
9 1

In [ ]:
<div data-v-490deea8="" class="geo-wrapper"><iframe data-v-490deea8="" data-v-666b8394="" src="https://www.google.com/maps/embed?pb=!4v1686332723639!6m8!1m7!1sCAoSLEFGMVFpcFBGMG5rYjFCcGhYNm5ObENGQmJHUEdHUXBWMmUwYjBSMXFBUEFT!2m2!1d55.73173347549965!2d37.50606995075941!3f268.32!4f19.040000000000006!5f0.4000000000000002" width="100%" height="100%" class="geo"></iframe></div>

In [ ]:
data-v-666b8394

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def search_google(query):
    # For Safari, you just need to specify the browser; no need for driver path
    driver = webdriver.Safari()

    # Open google.com
    driver.get('https://www.google.com')

    # Find the search bar using its name attribute value
    search_bar = driver.find_element(By.NAME, 'q')

    # Clear it, just in case there's something already there
    search_bar.clear()

    # Type the search query into the search bar
    search_bar.send_keys(query)

    # Submit the search query (You can also use Keys.RETURN)
    search_bar.submit()

    time.sleep(5)  # pause for 5 seconds to see the results

    # Close the browser
    driver.quit()

search_google("OpenAI")


In [1]:
!pip install stockfish

Defaulting to user installation because normal site-packages is not writeable


In [9]:
!pip3 install svg.path

Defaulting to user installation because normal site-packages is not writeable


In [46]:
!pip install python-chess

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 3.4 MB/s eta 0:00:00a 0:00:01
